In [1]:
%run init.ipynb

matchzoo version 1.1
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.models.DynamicClip.get_default_preprocessor()

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7900.39it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 4748.41it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 3191756.23it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9500.43it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5367.78it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 908655.75it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 9425.92it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f1854467080>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f1854fe1390>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [5]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [7]:
padding_callback = mz.models.Dynamic_Clip.get_default_padding_callback(
    fixed_length_left=10,
    fixed_length_right=100,
    pad_word_value=0,
    pad_word_mode='pre'
)

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [8]:
model = mz.models.Dynamic_Clip()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['clip_type'] = 'max'
model.params['left_length'] = 10
model.params['right_length'] = 100
model.params['hidden_dim'] = 128
model.params['kernel_size'] = 3
model.params['pool_size'] = 2
model.params['left_topk'] = 10
model.params['right_topk'] = 5

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

Dynamic_Clip(
  (embedding): Embedding(30058, 100, padding_idx=0)
  (dropout): Dropout(p=0.2, inplace=False)
  (attention): Dynamic_Clip_Attention()
  (conv_activation): ReLU()
  (hidden_activation): Tanh()
  (conv1d_left): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(100, 128, kernel_size=(3,), stride=(1,))
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv1d_right): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(100, 128, kernel_size=(3,), stride=(1,))
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (W): Linear(in_features=7040, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=1, bias=True)
)
Trainable params:  3984233


In [9]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [10]:
trainer.run()

[Iter-102 Loss-0.893]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4877 - normalized_discounted_cumulative_gain@5(0.0): 0.5511 - mean_average_precision(0.0): 0.516



[Iter-204 Loss-0.685]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6066 - normalized_discounted_cumulative_gain@5(0.0): 0.6637 - mean_average_precision(0.0): 0.63



[Iter-306 Loss-0.523]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6294 - normalized_discounted_cumulative_gain@5(0.0): 0.6886 - mean_average_precision(0.0): 0.646



[Iter-408 Loss-0.448]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6422 - normalized_discounted_cumulative_gain@5(0.0): 0.7007 - mean_average_precision(0.0): 0.661



[Iter-510 Loss-0.337]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6468 - normalized_discounted_cumulative_gain@5(0.0): 0.7048 - mean_average_precision(0.0): 0.6649



[Iter-612 Loss-0.287]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6832 - normalized_discounted_cumulative_gain@5(0.0): 0.731 - mean_average_precision(0.0): 0.6951



[Iter-714 Loss-0.229]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6735 - normalized_discounted_cumulative_gain@5(0.0): 0.7272 - mean_average_precision(0.0): 0.6902



[Iter-816 Loss-0.145]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6477 - normalized_discounted_cumulative_gain@5(0.0): 0.6956 - mean_average_precision(0.0): 0.6602



[Iter-918 Loss-0.126]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.64 - normalized_discounted_cumulative_gain@5(0.0): 0.6946 - mean_average_precision(0.0): 0.6521



[Iter-1020 Loss-0.105]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6608 - normalized_discounted_cumulative_gain@5(0.0): 0.7089 - mean_average_precision(0.0): 0.6703

Cost time: 1008.8958778381348s


In [11]:
model = mz.models.Dynamic_Clip()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['clip_type'] = 'threshold'
model.params['left_length'] = 10
model.params['right_length'] = 100
model.params['hidden_dim'] = 128
model.params['kernel_size'] = 3
model.params['pool_size'] = 2
model.params['left_threshold'] = 0.08
model.params['right_threshold'] = 0.08

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

Dynamic_Clip(
  (embedding): Embedding(30058, 100, padding_idx=0)
  (dropout): Dropout(p=0.2, inplace=False)
  (attention): Dynamic_Clip_Attention()
  (conv_activation): ReLU()
  (hidden_activation): Tanh()
  (conv1d_left): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(100, 128, kernel_size=(3,), stride=(1,))
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv1d_right): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(100, 128, kernel_size=(3,), stride=(1,))
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (W): Linear(in_features=7040, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=1, bias=True)
)
Trainable params:  3984233


In [12]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [13]:
trainer.run()

[Iter-102 Loss-0.882]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4665 - normalized_discounted_cumulative_gain@5(0.0): 0.5273 - mean_average_precision(0.0): 0.49



[Iter-204 Loss-0.688]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6077 - normalized_discounted_cumulative_gain@5(0.0): 0.6649 - mean_average_precision(0.0): 0.6251



[Iter-306 Loss-0.520]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6573 - normalized_discounted_cumulative_gain@5(0.0): 0.7066 - mean_average_precision(0.0): 0.666



[Iter-408 Loss-0.423]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6574 - normalized_discounted_cumulative_gain@5(0.0): 0.7138 - mean_average_precision(0.0): 0.6735



[Iter-510 Loss-0.329]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6576 - normalized_discounted_cumulative_gain@5(0.0): 0.708 - mean_average_precision(0.0): 0.666



[Iter-612 Loss-0.258]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6711 - normalized_discounted_cumulative_gain@5(0.0): 0.713 - mean_average_precision(0.0): 0.6673



[Iter-714 Loss-0.169]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.661 - normalized_discounted_cumulative_gain@5(0.0): 0.7066 - mean_average_precision(0.0): 0.6641



[Iter-816 Loss-0.156]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.672 - normalized_discounted_cumulative_gain@5(0.0): 0.7253 - mean_average_precision(0.0): 0.6795



[Iter-918 Loss-0.120]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6691 - normalized_discounted_cumulative_gain@5(0.0): 0.7161 - mean_average_precision(0.0): 0.6743



[Iter-1020 Loss-0.080]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6766 - normalized_discounted_cumulative_gain@5(0.0): 0.7328 - mean_average_precision(0.0): 0.6862

Cost time: 1008.7656784057617s
